# Capstone Workbook 4: Initial Modelling

In [1]:
# Import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import data 
airbnb_ldn = pd.read_csv('airbnb_ldn_final.csv')

The data has now been cleaned, had some initial EDA completed and been preprocessed. 

Some initial models will now be built, starting with a linear regression model.